In [27]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import pandas as pd
from scipy.special import softmax
import numpy as np

In [3]:
tokenizer = AutoTokenizer.from_pretrained("juliensimon/autonlp-song-lyrics-18753417")
model = AutoModelForSequenceClassification.from_pretrained("juliensimon/autonlp-song-lyrics-18753417")

tokenizer_config.json:   0%|          | 0.00/311 [00:00<?, ?B/s]

c:\Users\berzi\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\berzi\.cache\huggingface\hub\models--juliensimon--autonlp-song-lyrics-18753417. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/980 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/juliensimon/autonlp-song-lyrics-18753417/c0a1ada6918c4e8ac9ac1854d91e70a09ac4080a042ceb0a72e63db5a2660e44?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1713773425&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMzc3MzQyNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9qdWxpZW5zaW1vbi9hdXRvbmxwLXNvbmctbHlyaWNzLTE4NzUzNDE3L2MwYTFhZGE2OTE4YzRlOGFjOWFjMTg1NGQ5MWU3MGEwOWFjNDA4MGEwNDJjZWIwYTcyZTYzZGI1YTI2NjBlNDQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=FvYpHrL%7EcOEstVT5y3tsKFXJmcEHnrI0-RoDNzCjYyXO%7EnwxG86EMmPjA%7E0ITHvlIleXK6aa2naBdLagq%7E3CB2FYdISSkJz3yEuCd7IjLfwkHYbBUzVB9Tmckg-NyzC14hnPIVWeU8f-4X7db%7EInvEZo8WuuCW76Yz1V90FtI7xwfAO0MgGC1zcbDeXo02Nplgi3d18HVeWPKed0uItb4OIHsf7vvRrhouL4M8Ap7BhEtDjNUY1D9Rcu1ZvsGaCWt3NbmC1O5KwhSiU5l29K2UX%7ELFfwuidOmuZX2XPX4HCakRrB5I5EH9TShn

model.safetensors:  34%|###3      | 147M/438M [00:00<?, ?B/s]

In [21]:
config = AutoConfig.from_pretrained("juliensimon/autonlp-song-lyrics-18753417")

In [84]:
df = pd.read_csv("data/processed_spotify_millsongdata.csv")

In [85]:
df = df.head(100).copy()

In [86]:
def get_genres(text):
    try:
        inputs = tokenizer(text,  return_tensors="pt")
        outputs = model(**inputs)

        scores = outputs[0][0].detach().numpy()
        scores = softmax(scores)
        ranking = np.argsort(scores)
        ranking = ranking[::-1]

        genres = []
        for i in range(scores.shape[0]):
            l = config.id2label[ranking[i]]
            s = scores[ranking[i]]
            if s >= 0.1:
                genres.append(l)
                # print(f"{i+1}) {l} {np.round(float(s), 4)}")
    except RuntimeError:
        genres = ['Unknown']
        

    return ','.join(genres)

In [87]:
df['genres'] = df["cleaned_text"].apply(lambda x: get_genres(x))

In [88]:
df.head()

,artist,song,cleaned_text,genres
0,ABBA,Ahe's My Kind Of Girl,"look at her face , it 's a wonderful face and ...",Rock
1,ABBA,"Andante, Andante","take it easy with me , please touch me gently ...","Pop,Dance,Rock"
2,ABBA,As Good As New,i 'll never know why i had to go why i had to ...,"Pop,Rock"
3,ABBA,Bang,making somebody happy is a question of give an...,Pop
4,ABBA,Bang-A-Boomerang,making somebody happy is a question of give an...,"Pop,Rock"
